In [1]:
from dotenv import load_dotenv
load_dotenv()
# get openai-key
import os
from autogen import ConversableAgent
openai_key = os.getenv('OPENAI_API_KEY')
llm_config={"model":"gpt-3.5-turbo"}

In [2]:
llm_config = {"model": "gpt-3.5-turbo"}

task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''

create Writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])
print(reply)

Title: "Unleashing the Power of Deep Learning with DeepLearning.AI"

Discover the magic of DeepLearning.AI, a beacon of excellence in the world of artificial intelligence and deep learning. Founded by the renowned Andrew Ng, this platform offers top-tier courses, specialization programs, and resources to fuel your passion for AI. Whether you're a seasoned professional or a curious learner, DeepLearning.AI equips you with the knowledge and skills needed to thrive in this dynamic field. From computer vision to natural language processing, dive into the endless possibilities of deep learning and unlock a world of innovation with DeepLearning.AI by your side.


add reflection

In [4]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing the Power of Deep Learning with DeepLearning.AI"

Discover the magic of DeepLearning.AI, a beacon of excellence in the world of artificial intelligence and deep learning. Founded by the renowned Andrew Ng, this platform offers top-tier courses, specialization programs, and resources to fuel your passion for AI. Whether you're a seasoned professional or a curious learner, DeepLearning.AI equips you with the knowledge and skills needed to thrive in this dynamic field. From computer vision to natural language processing, dive into the endless possibilities of deep learning and unlock a world of innovation with DeepLearning.AI by your side.

---------------------------------------------------------------------------

nested Conversation

![nested Conversation](Image/image3.png)

In [5]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

nested conversation

In [7]:
# Gather summary from the agents in the ato chat session
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]

In [8]:
critic.register_nested_chats(
    review_chats,
    trigger=writer, # 
)

res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

print(res.summary)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing the Power of Deep Learning with DeepLearning.AI"

Discover the magic of DeepLearning.AI, a beacon of excellence in the world of artificial intelligence and deep learning. Founded by the renowned Andrew Ng, this platform offers top-tier courses, specialization programs, and resources to fuel your passion for AI. Whether you're a seasoned professional or a curious learner, DeepLearning.AI equips you with the knowledge and skills needed to thrive in this dynamic field. From computer vision to natural language processing, dive into the endless possibilities of deep learning and unlock a world of innovation with DeepLearning.AI by your side.

---------------------------------------------------------------------------